In [6]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score


## Result read csv


In [5]:
result = pd.read_csv('~/Desktop/2024_UCLA/Spring/418/Project/result.csv')
result.columns


Index(['url', 'min_price', 'max_price', 'EVENT_ID', 'EVENT_NAME',
       'EVENT_NOTES', 'EVENT_INFO', 'EVENT_STATUS', 'EVENT_START_DATETIME',
       'EVENT_END_DATETIME', 'EVENT_START_LOCAL_DATE',
       'EVENT_START_LOCAL_TIME', 'ONSALE_START_DATETIME',
       'ONSALE_END_DATETIME', 'CLASSIFICATION_SEGMENT', 'CLASSIFICATION_GENRE',
       'CLASSIFICATION_SUB_GENRE', 'EVENT_IMAGE_URL', 'MIN_PRICE', 'MAX_PRICE',
       'CURRENCY', 'ATTRACTION_NAME', 'ATTRACTION_ID', 'ATTRACTION_IMAGE_URL',
       'VENUE_NAME', 'VENUE_ID', 'VENUE_STREET', 'VENUE_CITY',
       'VENUE_STATE_CODE', 'VENUE_COUNTRY_CODE', 'VENUE_LATITUDE',
       'VENUE_LONGITUDE', 'VENUE_ZIP_CODE', 'VENUE_TIMEZONE', 'ATTRACTION_URL',
       'VENUE_URL', 'PRIMARY_EVENT_URL', 'RESALE_EVENT_URL', 'PRESALE_NAME',
       'PRESALE_DATETIME_RANGE', 'LEGACY_EVENT_ID', 'LEGACY_VENUE_ID',
       'LEGACY_ATTRACTION_ID', 'PRESALE_NAME.1', 'PRESALE_DESCRIPTION',
       'PRESALE_START_DATETIME', 'PRESALE_END_DATETIME', 'SOURCE',
       'C

In [8]:
result['average_price'] = result[['min_price','max_price']].mean(axis=1)


In [33]:
result['HOT_EVENT'] = result['HOT_EVENT'].astype(int) 
# result['HOT_EVENT'] = result['HOT_EVENT'].apply(lambda x: 1 if x == True else 0)
result['HOT_EVENT'].value_counts()


HOT_EVENT
0    703
1    115
Name: count, dtype: int64

In [34]:
features = [
    'CLASSIFICATION_SEGMENT', 
    'CLASSIFICATION_GENRE',
    'HOT_EVENT',
    'VENUE_CITY', 
    'VENUE_STATE_CODE', 
    'VENUE_ZIP_CODE',
    'time_to_start',# how long it takes until the event kicks off
    'time_since_start_sale', # how long has it been since the start sale
    'time_to_end_sale'
]
X = result[features]
y = result['average_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features = ['CLASSIFICATION_SEGMENT', 'CLASSIFICATION_GENRE', 'HOT_EVENT', 'VENUE_CITY', 'VENUE_STATE_CODE', 'VENUE_ZIP_CODE']



In [35]:
for col in categorical_features:
    X_train[col] = X_train[col].astype(str).fillna('missing')
    X_test[col] = X_test[col].astype(str).fillna('missing')


In [36]:
from catboost import CatBoostRegressor, Pool
model = CatBoostRegressor(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    cat_features=categorical_features,
    verbose=100
)

model.fit(X_train, y_train)

0:	learn: 83.3074272	total: 20ms	remaining: 1.98s
99:	learn: 46.5515843	total: 150ms	remaining: 0us


In [37]:
from sklearn.metrics import mean_squared_error, r2_score

# 预测
y_pred = model.predict(X_test)

# 计算 MSE 和 R²
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R²): {r2:.4f}")


Mean Squared Error (MSE): 2940.29
R-squared (R²): 0.5192
